In [1]:
# Import SlackerGPU to set env variables, import tf (tf 1.4)
from handcam.ltt.util import SlackerGPU
slackerGPU = SlackerGPU.SlackerGPU(username='ltaverne',
                                   desired_server='ait-server-03',
                                   num_gpus=1)
import tensorflow as tf

try:
    del flags
except NameError:
    pass
try:
    del FLAGS
except NameError:
    pass

flags = tf.app.flags

#State your dataset directory
flags.DEFINE_string('dataset_dir', '/local/home/luke/datasets/rgbd-dataset/', 'String: Your dataset directory')
flags.DEFINE_string('mode', 'train', 'train or eval.')
flags.DEFINE_integer('image_size', 224, 'Image side length.')
flags.DEFINE_integer('batch_size', 10, 'Batch size.')
flags.DEFINE_integer('num_classes', 51, 'Number of classes.')
flags.DEFINE_string('train_dir', '/tmp/luke/WRN/train',
                           'Directory to keep training outputs.')
flags.DEFINE_string('eval_dir', '/tmp/luke/WRN/eval',
                           'Directory to keep eval outputs.')
flags.DEFINE_integer('eval_batch_count', 10,
                            'Number of batches to eval.')
flags.DEFINE_bool('eval_once', False,
                         'Whether evaluate the model only once.')
flags.DEFINE_string('log_root', '/tmp/luke/WRN',
                           'Directory to keep the checkpoints. Should be a '
                           'parent directory of FLAGS.train_dir/eval_dir.')

# The number of images in the validation set. You would have to know the total number of examples in advance. This is essentially your evaluation dataset.
flags.DEFINE_float('validation_size', 0.1, 'Float: The proportion of examples in the dataset to be used for validation')

# The number of shards per dataset split.
flags.DEFINE_integer('num_shards', 1000, 'Int: Number of shards to split the TFRecord files')

# Seed for repeatability.
flags.DEFINE_integer('random_seed', 0, 'Int: Random seed to use for repeatability.')

#Output filename for the naming the TFRecord file
flags.DEFINE_string('tfrecord_filename', 'uw-rgbd', 'String: The output filename to name your TFRecord file')

FLAGS = flags.FLAGS

SlackerGPU: Using gpus [7] on server: ait-server-03


/local/home/luke/.virtualenvs/mkl_dnn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
import numpy as np
import os
import sys
import six
import time
from handcam.ltt.network.model.Wide_ResNet import wide_resnet_tf as resnet_model



/local/home/luke/.virtualenvs/mkl_dnn/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


IndentationError: unexpected indent (Preprocessing.py, line 98)

In [3]:
def int64_feature(values):
    """Returns a TF-Feature of int64s.
    Args:
    values: A scalar or list of values.
    Returns:
    a TF-Feature.
    """
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))


def bytes_feature(values):
    """Returns a TF-Feature of bytes.
    Args:
    values: A string.
    Returns:
    a TF-Feature.
    """
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def _merge_rgb_depth(rgb, depth, label):
    im = tf.concat([tf.cast(rgb,dtype=tf.float32), tf.cast(depth,dtype=tf.float32)], axis=2)
    
    # im = tf.transpose(im,[2,0,1])
    im = tf.image.resize_image_with_crop_or_pad(im, 224, 224)
    
    return im, label

def _parse_function(example_proto):
    features = {"image/rgb": tf.FixedLenFeature((), tf.string, default_value=""),
                "image/depth": tf.FixedLenFeature((), tf.string, default_value=""),
              "image/class/label": tf.FixedLenFeature((), tf.int64, default_value=0)}
    parsed_features = tf.parse_single_example(example_proto, features)
    
    rgb_img = tf.image.decode_image(parsed_features["image/rgb"], channels=3)
    # rgb_img.set_shape([None, 320, 640, 3])
    
    depth_img = tf.image.decode_image(parsed_features["image/depth"], channels=1)
    # depth_img.set_shape([None, 320, 640, 1])
    
    one_hot = tf.one_hot(parsed_features["image/class/label"],len(class_names))
    
    # Flip
    # rgb_img, depth_img, one_hot = _random_flip_lr(rgb_img, depth_img, one_hot)
    
    # Translate
    # rgb_img, depth_img, one_hot = _random_translate(rgb_img, depth_img, one_hot)
    
    # Rotate
    # rgb_img, depth_img, one_hot = _random_rotations(rgb_img, depth_img, one_hot)    
    
    # Merge
    img, one_hot = _merge_rgb_depth(rgb_img, depth_img, one_hot)
    
    return img, one_hot

def _random_rotations(rgb, depth, label):
    rotate = np.random.rand()
    
    if rotate > 0.5:
        angle = (-0.174 - 1.66)*np.random.rand() + 1.66 # Between -95 and 10 degrees
        old_shape_rgb = rgb.shape
        old_shape_depth = rgb.shape
        # rgb.set_shape([480, 640, 3])
        # depth.set_shape([480, 640, 1])
        rgb = tf.contrib.image.rotate(rgb, angle)
        depth = tf.contrib.image.rotate(depth, angle)

        # rgb.set_shape(old_shape_rgb)
        # depth.set_shape(old_shape_depth)

        
    return rgb, depth, label

def _random_flip_lr(rgb, depth, label):
    flip = np.random.rand()
    
    if flip > 0.5:
        rgb = tf.image.flip_left_right(rgb)
        depth = tf.image.flip_left_right(depth)
        
    return rgb, depth, label

def _random_translate(rgb, depth, label):
    translate = np.random.rand()
    
    # if translate > 0.5:
    if True:
        old_shape_rgb = rgb.shape
        old_shape_depth = rgb.shape
        rgb.set_shape([480, 640, 3])
        depth.set_shape([480, 640, 1])
        
        transforms = [1, 0, 0, 0, 1, 0, 0, 0]
        
        transforms[2] = np.random.randint(-45,45) # x shift
        transforms[5] = np.random.randint(-45,45) # y shift
        
        rgb = tf.contrib.image.transform(rgb, transforms)
        depth = tf.contrib.image.transform(depth, transforms)
        # 
        # rgb.set_shape(old_shape_rgb)
        # depth.set_shape(old_shape_depth)
        
    return rgb, depth, label

In [4]:
# Setup the dataset for uwrgbd
dataset_root = FLAGS.dataset_dir
class_names = sorted([i for i in next(os.walk(dataset_root))[1]])
class_names_to_index = dict(zip(class_names,range(len(class_names))))

test_tfrecord_path = FLAGS.dataset_dir + 'uw-rgbd_train_00000-of-00900.tfrecord'

filenames_placeholder = tf.placeholder(tf.string, shape=[None])

dataset = tf.data.TFRecordDataset(filenames_placeholder)
dataset = dataset.map(_parse_function, num_parallel_calls=4)

dataset = dataset.repeat()
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(FLAGS.batch_size))
# dataset = dataset.map(lambda x, y: (x.set_shape([16, 480,640, 4]), y))
dataset = dataset.prefetch(1)

# iterator = dataset.make_initializable_iterator()
# iterator = dataset.make_one_shot_iterator()

iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init_train = iterator.make_initializer(dataset)
images, labels = iterator.get_next()

train_tfrecord_filenames = [test_tfrecord_path]

# sess = tf.Session('')
# sess.run(iterator.initializer, feed_dict={filenames_placeholder: train_tfrecord_filenames})
# 
# images, labels = iterator.get_next()


In [5]:

class _DatasetInitializerHook(tf.train.SessionRunHook):
    def __init__(self, initializer, filenames_list):
        self._initializer = initializer
        self._filenames_list = filenames_list
    def begin(self):
        pass
    def after_create_session(self, session, coord):
        del coord
        session.run(self._initializer, feed_dict={filenames_placeholder: self._filenames_list})
            
def train(hps):
    """Training loop."""
    # images, labels = sess.run(next_element)
    # images_placeholder = tf.placeholder(tf.float32, shape=[FLAGS.batch_size, 480, 640, 4], name='images_placeholder')
    # labels_placeholder = tf.placeholder(tf.float32, shape=[FLAGS.batch_size, FLAGS.num_classes], name='labels_placeholder')
    model = resnet_model.ResNet(hps, images, labels, FLAGS.mode, batch_size=FLAGS.batch_size)
    model.build_graph()
    

    param_stats = tf.contrib.tfprof.model_analyzer.print_model_analysis(
        tf.get_default_graph(),
        tfprof_options=tf.contrib.tfprof.model_analyzer.
        TRAINABLE_VARS_PARAMS_STAT_OPTIONS)
    sys.stdout.write('total_params: %d\n' % param_stats.total_parameters)

    tf.contrib.tfprof.model_analyzer.print_model_analysis(
        tf.get_default_graph(),
        tfprof_options=tf.contrib.tfprof.model_analyzer.FLOAT_OPS_OPTIONS)

    truth = tf.argmax(model.labels, axis=1)
    predictions = tf.argmax(model.predictions, axis=1)
    precision = tf.reduce_mean(tf.to_float(tf.equal(predictions, truth)))

    summary_hook = tf.train.SummarySaverHook(
        save_steps=100,
        output_dir=FLAGS.train_dir,
        summary_op=tf.summary.merge([model.summaries,
                                     tf.summary.scalar('Precision', precision)]))

    logging_hook = tf.train.LoggingTensorHook(
        tensors={'step': model.global_step,
                 'loss': model.cost,
                 'precision': precision},
        every_n_iter=100)

    class _LearningRateSetterHook(tf.train.SessionRunHook):
        """Sets learning_rate based on global step."""

        def begin(self):
            self._lrn_rate = 0.1

        def before_run(self, run_context):
            return tf.train.SessionRunArgs(
                model.global_step,  # Asks for global step value.
                feed_dict={model.lrn_rate: self._lrn_rate})  # Sets learning rate

        def after_run(self, run_context, run_values):
            train_step = run_values.results
            if train_step < 40000:
                self._lrn_rate = 0.1
            elif train_step < 60000:
                self._lrn_rate = 0.01
            elif train_step < 80000:
                self._lrn_rate = 0.001
            else:
                self._lrn_rate = 0.0001
                
    initializer_hook = _DatasetInitializerHook(init_train, train_tfrecord_filenames)

    with tf.train.MonitoredTrainingSession(
            checkpoint_dir=FLAGS.log_root,
            hooks=[initializer_hook, logging_hook, _LearningRateSetterHook()],
            chief_only_hooks=[summary_hook],
            # Since we provide a SummarySaverHook, we need to disable default
            # SummarySaverHook. To do that we set save_summaries_steps to 0.
            save_summaries_steps=0,
            config=tf.ConfigProto(allow_soft_placement=True)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(model.train_op)
            


def evaluate(hps):
    """Eval loop."""
    model = resnet_model.ResNet(hps, images, labels, FLAGS.mode, batch_size=FLAGS.batch_size)
    model.build_graph()
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter(FLAGS.eval_dir)

    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init_train, feed_dict={filenames_placeholder: train_tfrecord_filenames})
    # tf.train.start_queue_runners(sess)

    best_precision = 0.0
    while True:
        try:
            ckpt_state = tf.train.get_checkpoint_state(FLAGS.log_root)
        except tf.errors.OutOfRangeError as e:
            tf.logging.error('Cannot restore checkpoint: %s', e)
            continue
        if not (ckpt_state and ckpt_state.model_checkpoint_path):
            tf.logging.info('No model to eval yet at %s', FLAGS.log_root)
            continue
        tf.logging.info('Loading checkpoint %s', ckpt_state.model_checkpoint_path)
        saver.restore(sess, ckpt_state.model_checkpoint_path)

        total_prediction, correct_prediction = 0, 0
        for _ in six.moves.range(FLAGS.eval_batch_count):
            (summaries, loss, predictions, truth, train_step) = sess.run(
                [model.summaries, model.cost, model.predictions,
                 model.labels, model.global_step])

            truth = np.argmax(truth, axis=1)
            predictions = np.argmax(predictions, axis=1)
            correct_prediction += np.sum(truth == predictions)
            total_prediction += predictions.shape[0]

        precision = 1.0 * correct_prediction / total_prediction
        best_precision = max(precision, best_precision)

        precision_summ = tf.Summary()
        precision_summ.value.add(
            tag='Precision', simple_value=precision)
        summary_writer.add_summary(precision_summ, train_step)
        best_precision_summ = tf.Summary()
        best_precision_summ.value.add(
            tag='Best Precision', simple_value=best_precision)
        summary_writer.add_summary(best_precision_summ, train_step)
        summary_writer.add_summary(summaries, train_step)
        tf.logging.info('loss: %.3f, precision: %.3f, best precision: %.3f' %
                        (loss, precision, best_precision))
        summary_writer.flush()

        if FLAGS.eval_once:
            break

        time.sleep(60)


def main(_):
    hps = resnet_model.HParams(batch_size=FLAGS.batch_size,
                               num_classes=FLAGS.num_classes,
                               min_lrn_rate=0.0001,
                               lrn_rate=0.1,
                               num_residual_units=5,
                               use_bottleneck=False,
                               weight_decay_rate=0.0002,
                               relu_leakiness=0.1,
                               optimizer='mom')

    if FLAGS.mode == 'train':
        train(hps)
    elif FLAGS.mode == 'eval':
        evaluate(hps)


In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
tf.app.run()

(10, 64)
(10, 64)


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details


627 ops no flops stats due to incomplete shapes.


total_params: 466963


...

INFO:tensorflow:global_step/sec: 1.60414


INFO:tensorflow:precision = 1.0, loss = 0.1199385, step = 21674 (62.376 sec)


INFO:tensorflow:global_step/sec: 1.70765


INFO:tensorflow:precision = 1.0, loss = 0.1255741, step = 21774 (58.523 sec)


KeyboardInterrupt: 